In [ ]:
# Filter features based of postfix
def columns_by_postfix(columns, postfix):
    result = [col for col in columns if postfix in col]
    result.sort()
    return result

def league_and_non_league_columns(columns, suffix):
    suf_columns = columns_by_postfix(columns, suffix)
    leagues = columns_by_postfix(suf_columns, 'league')
    non_leagues = [col for col in suf_columns if col not in leagues]
    return non_leagues, leagues

In [ ]:
# Divide columns list by mid index
def divide_columns(frame, suffix, postfix, is_league=False):
    if not is_league:
        cols = league_and_non_league_columns(frame.columns, suffix)[0]
    else:
        cols = league_and_non_league_columns(frame.columns, suffix)[1]
    selected_cols = columns_by_postfix(cols, postfix)
    mid = len(selected_cols) // 2
    features_1 = frame[selected_cols[:mid]]
    features_1.index = range(len(features_1))
    features_2 = frame[selected_cols[mid:]]
    features_2.index = range(len(features_2))
    return features_1, features_2

In [ ]:
# Boxplot for multiple features in dataframe
# Builds 2 plots for better readability
def multiple_boxplot(f_1, f_2):
    fig, ax = plt.subplots(ncols=2, figsize=(20, 14))
    flier_props = dict(markerfacecolor='r', marker='o')
    median_props = dict(linewidth=2.5, color='r')
    box_props = dict(linewidth=3, color='darkgoldenrod')
    mean_props = dict(marker='D', markerfacecolor='blue', markersize=12)
    bxp_list = []
    xlabels = []
    for (ax, f) in zip(ax, [f_1, f_2]):
        bxp_dict = ax.boxplot(f.to_numpy(), labels=f.columns, showmeans=True,
                              meanprops=mean_props, flierprops=flier_props,
                              boxprops=box_props, medianprops=median_props,
                              widths=0.75)        
        plt.draw()
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
        bxp_list.append(bxp_dict)
        xlabels.append(ax.get_xticklabels())
    plt.subplots_adjust(wspace=0.1)
    return bxp_list, xlabels

In [ ]:
def multiple_distplot(frame, features):
    fig, axes = plt.subplots(len(features), figsize=(20, 8))
    for (f, ax) in zip(features, axes):
        sns.distplot(frame[f], bins=20, color='blue', label=f,
                     fit=stats.norm, kde=False, ax=ax)
        
        chart_box = ax.get_position()
        ax.set_position([chart_box.x0, chart_box.y0,
                         chart_box.width, chart_box.height*0.8])
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.1),
                  shadow=True, ncol=1, fontsize=12)
        ax.grid(which='major', color='grey')

In [ ]:
pp_ranges = ['90_', '180_', '540_', '1350_', '2700_', '5400_', '8100_', '10800_']
pg_ranges = ['1_', '2_', '6_', '15_', '30_', '60_', '90_', '120_']
league_ranges = ['10_', '30_', '75_', '150_', '300_', '450_', '600_']

nl_cols, l_cols = league_and_non_league_columns(df.columns, 'sma')

pp_cols = columns_by_postfix(nl_cols, 'pp')
pg_cols = columns_by_postfix(nl_cols, 'pg')
lpp_cols = columns_by_postfix(l_cols, 'pp')
lpg_cols = columns_by_postfix(l_cols, 'pg')

pp_cols_by_range = [columns_by_postfix(pp_cols, rn) for rn in pp_ranges]
pg_cols_by_range = [columns_by_postfix(pg_cols, rn) for rn in pg_ranges]
lpp_cols_by_range = [columns_by_postfix(lpp_cols, rn) for rn in league_ranges]
lpg_cols_by_range = [columns_by_postfix(lpg_cols, rn) for rn in league_ranges]

In [ ]:
# Plot multiple plots
# postfix - can has only two values: 'pp' and 'pg'
# is_league - indicates whether we want to plot league columns or not
def multiple_plot(frame, suffix, postfix, colors, from_index,
                  to_index=None, is_league=False):
    if not is_league:
        cols = league_and_non_league_columns(frame.columns, suffix)[0]
    else:
        cols = league_and_non_league_columns(frame.columns, suffix)[1]
    selected_cols = columns_by_postfix(cols, postfix)
    if to_index is None:
        features = frame[selected_cols[from_index:]]
    else:
        features = frame[selected_cols[from_index:to_index]]
    features.index = range(len(features))
    
    fig, ax = plt.subplots(figsize=(20, 15))
    for (column, color) in zip(features.columns, colors):
        ax.plot(features.loc[:, column] * 10, color=color, label=column)
    ax.plot(frame['fp0'] * 0.01, color='#ab3f48', label='fp0', linewidth=3)
    legend = ax.legend(loc='upper left', shadow=True, fontsize=12)
    legend.get_frame().set_facecolor('w')

In [ ]:
colors = ['b', 'g', 'r', 'y', 'c', 'm', 'k', '#bc9fdd', '#d3d3d3']

# Strike outs
#multiple_plot(sample, 'hr', 'pp', colors, 0, 4)
#multiple_plot(sample, 'hr', 'pp', colors, 4)